# Adding information to BdHNe table

One of these days I published the online version of ICRANet Binary-driven Hyper-Novae table of Gamma-Ray Bursts (GRB) at the (BSDC/VO)[http://vo.bsdc.icranet.org] website; the table: http://vo.bsdc.icranet.org/bdhne/q/web/form.
The (current) first version of the table contains exactly what is being published in the article (Ruffini et al, in preparation), which is:

* GRB (id): the standard name for GRBs (*YYMMDD*[A-Z]);
* Designation: as proposed by the authors of the article;
* z: redshift;
* E_iso: the energy computedby the authors after their model;
* Instrument: observatory from where data was used;
* GCN: the GCN corresponding to the event.

What I'm going to do now is to add position (Right Ascension and Declination) and trigger time of those events.
To do so, I should retrieve such information from online services/tables and join them with the current table.

Swift GRBs data will be used to get such information (position and trigger time). 
(*"--Why Swift? --Because I had to choose one; next time will be Fermi."*)
After a search for which website could give me better the information, the [Mission's American website][NASAGRB].
Through their service I can easily automate the query, retrieve and parsing process, plus they provide all the relevant information I am looking for (almost) all GRBs.
For the records, below is the list of websites I looked into; The list was kindly provided by my colleaghe Milos Kovacevic:

* http://www.swift.ac.uk/xrt_positions/
* https://heasarc.gsfc.nasa.gov/W3Browse/fermi/fermigbrst.html
* https://heasarc.gsfc.nasa.gov/W3Browse/all/batsegrb.html
* http://www.astro.caltech.edu/grbox/grbox.php
* (**Chosen**) https://swift.gsfc.nasa.gov/archive/grb_table/

Data and software we already have (properly working/formatted):

* The current BdHNe table is here copied as filename `ibdhne.csv`;
* The Perl script to retrieve the table from [Swift GRB service][NASAGRB], called `SwiftGRBsNASA.pl`;
 * We will execute the script to generate an up-to-date table of GRBs named `SwiftGRBsNASA_table.csv`.

[NASAGRB]: https://swift.gsfc.nasa.gov/archive/grb_table/

In [1]:
import pandas
pandas.set_option('display.max_columns',None)

Output will go in 'tab' dataframe. The output table will contain then the current columns of `ibdhne` plus
position -- `ra`, `dec`, `error radius` -- and `trigger time`.

In [2]:
# to-be-filled output structure
tab = pandas.DataFrame()

## The IBdHNe table

In [3]:
ibd = pandas.read_csv('data.csv',sep=';')
print ibd

         GRB     designation      z   E_iso instrument    GCN
0    050315A  IBdHNe 050315A  1.950    8.32      Swift   3099
1    050318A  IBdHNe 050318A  1.440    3.70      Swift   3134
2    050319A  IBdHNe 050319A  3.240   20.20      Swift   3119
3    050401A  IBdHNe 050401A  2.900   92.00         KW   3179
4    050408A  IBdHNe 050408A  1.240   10.80       HETE   3188
5    050505A  IBdHNe 050505A  4.270   44.60      Swift   3364
6    050525A  IBdHNe 050525A  0.606    2.31         KW   3474
7    050730A  IBdHNe 050730A  3.970   42.80      Swift   3715
8    050802A  IBdHNe 050802A  1.710    7.46      Swift   3737
9    050814A  IBdHNe 050814A  5.300   26.80      Swift   3803
10   050820A  IBdHNe 050820A  2.610   39.00         KW   3852
11   050922C  IBdHNe 050922C  2.200   19.80         KW   4030
12   051109A  IBdHNe 051109A  2.350   18.30         KW   4238
13   060108A  IBdHNe 060108A  2.030    1.50      Swift   4445
14   060115A  IBdHNe 060115A  3.530   19.00      Swift   4518
15   060

## Swift GRBs table

In [4]:
grbstable = 'SwiftGRBsNASA_table.csv'
import os
if not os.path.exists(grbstable):
    with open(grbstable,'w') as fpout:
        import subprocess
        import shlex
        ret = subprocess.check_call( shlex.split('perl SwiftGRBsNASA.pl'), stdout=fpout, stderr=subprocess.STDOUT)

In [5]:
swf = pandas.read_csv(grbstable, sep=';')
print swf.head(10)

       GRB    Time|[UT]   Trigger|Number BAT RA|(J2000) BAT Dec|(J2000)  \
0  170325A  07:56:57.95  Ground Analysis        127.483          20.526   
1  170318B     15:27:52           743086        284.229           6.335   
2  170318A     12:11:56           743065        305.662          28.408   
3  170317A     09:45:59           742866         93.087          50.500   
4  170311A     08:08:42           741965        280.539         -30.041   
5  170307A     20:24:21           741528         13.510           9.538   
6  170306A     07:06:12           741220        263.087         -44.765   
7  170208B     22:33:38           737463        127.145          -9.027   
8  170208A     18:11:16           737438        166.544         -46.786   
9  170206B     21:57:25           737125        271.784          11.175   

  BAT 90%|Error Radius|[arcmin] BAT T90|[sec]  \
0                           2.0           0.3   
1                           3.7         160.0   
2                          

Regarding our goal -- to recover position and trigger time for each GRB -- we promptly see the `Time|[UT]` column, *RA*, *Dec* and the *error radius* given by `BAT`, `XRT` and `UVOT` instruments.

The *Time* column alone is incomplete since it does not inform the day the event happened; but this information is easily extracted from the GRB name itself.

Positional information comes in three flavors (*BAT*, *XRT*, *UVOT*), where *error radius* is the relevant information to decide which one to use; *BAT* has the worse precision (~`arcmin`) but has measurements for every GRB, while *XRT* and *UVOT* have much better measurements (~`arcsec`) but are followed less events.

By all means, let us work a little bit over the data in table itself.
For instance, let us handle the *Not Availables* (`n/a`) and then look some stats.

### GRB table pre-processing: fixing datatypes and finding nulls

In [6]:
# First, let's have a glance on columns values; let's try to figure out which one has "nulls".
#
# One way of doing this is by looking the frequency of values -- if a given value is two frequent, probably 
# that is the value is being used as 'NA'(i.e, Null) for that column (this comes with the assumption that such 
# table has a somewhat broad distribution of values);
#
# Before going further, what are columns datatypes?
print swf.dtypes

GRB                                              object
Time|[UT]                                        object
Trigger|Number                                   object
BAT RA|(J2000)                                   object
BAT Dec|(J2000)                                  object
BAT 90%|Error Radius|[arcmin]                    object
BAT T90|[sec]                                    object
BAT Fluence|(15-150 keV)|[10-7 erg/cm2]          object
XRT RA|(J2000)                                   object
XRT Dec|(J2000)                                  object
XRT 90%|Error Radius|[arcsec]                    object
XRT Time to First|Observation|[sec]              object
XRT Early Flux|(0.3-10 keV)|[10-11 erg/cm2/s]    object
UVOT RA|(J2000)                                  object
UVOT Dec|(J2000)                                 object
UVOT 90%|Error Radius|[arcsec]                   object
UVOT Time to|First Observation|[sec]             object
UVOT Magnitude                                  

In [7]:
# So, what are the frequency of values we find at each column?
#
# I am assuming that (1) few (correct) values are going to show up more than once, simply because we are dealing
# with real number and free-form comments; in principle, there is no restriction on the values.
# Following that, I assume that Null values will be among the most frequent values if not the only ones in the
# columns they occur.
freqs = pandas.DataFrame( {col:swf[col].value_counts().head().index.values for col in swf.columns} )
print freqs

  BAT 90%|Error Radius|[arcmin] BAT Dec|(J2000)  \
0                           1.0         -26.667   
1                           1.1         -44.177   
2                           1.2          13.039   
3                           1.8           9.139   
4                           1.7         -26.146   

  BAT Fluence|(15-150 keV)|[10-7 erg/cm2] BAT RA|(J2000) BAT T90|[sec]  \
0                                     n/a        241.273           n/a   
1                                      14        144.521            48   
2                                      13        282.338            64   
3                                      12        120.198          48.0   
4                                      11        270.992          11.0   

                                            Comments      GRB  \
0                                 UVOT: no detection  150821A   
1               XRT: no detection|UVOT: no detection   071129   
2  No AT slew (Earth limb constraint)|UVOT: no de... 

We see then that some columns do present Null values and they are all '**n/a**'.
We should be able to mask them very easily.

In [8]:
# Mask 'n/a' values
#
for col in swf.columns:
    to_replace = swf[col] == 'n/a'
    swf.loc[to_replace,col] = None
freqs = pandas.DataFrame( {col:swf[col].value_counts().head().index.values for col in swf.columns} )
print freqs

  BAT 90%|Error Radius|[arcmin] BAT Dec|(J2000)  \
0                           1.0         -26.667   
1                           1.1         -44.177   
2                           1.2          13.039   
3                           1.8           9.139   
4                           1.7         -26.146   

  BAT Fluence|(15-150 keV)|[10-7 erg/cm2] BAT RA|(J2000) BAT T90|[sec]  \
0                                      14        241.273            48   
1                                      13        355.412            64   
2                                      12        270.992          48.0   
3                                      11         92.387            15   
4                                      15         13.078           0.3   

                                            Comments      GRB  \
0                                 UVOT: no detection  150821A   
1               XRT: no detection|UVOT: no detection   071129   
2  No AT slew (Earth limb constraint)|UVOT: no de... 

Nulls are properly flagged/masked now. This is important because we can now handle the data with confidence.
For instance, let's see which columns have nils in it.

In [9]:
from termcolor import colored
print "Has null?"
for col in swf.columns:
    hasnil = any(swf[col].isnull())
    col = colored(col,on_color='on_red') if hasnil else colored(col,'blue')
    print '  {}: {}'.format(col,hasnil)

Has null?
  GRB: False
  Time|[UT]: True
  Trigger|Number: False
  BAT RA|(J2000): True
  BAT Dec|(J2000): True
  BAT 90%|Error Radius|[arcmin]: True
  BAT T90|[sec]: True
  BAT Fluence|(15-150 keV)|[10-7 erg/cm2]: True
  XRT RA|(J2000): True
  XRT Dec|(J2000): True
  XRT 90%|Error Radius|[arcsec]: True
  XRT Time to First|Observation|[sec]: True
  XRT Early Flux|(0.3-10 keV)|[10-11 erg/cm2/s]: True
  UVOT RA|(J2000): True
  UVOT Dec|(J2000): True
  UVOT 90%|Error Radius|[arcsec]: True
  UVOT Time to|First Observation|[sec]: True
  UVOT Magnitude: True
  Other Observatory Detections: True
  Redshift: True
  Host Galaxy: True
  Comments: True
  References: False


In [10]:
# Now let's fix/transform columns to formats we like better.
#
# For instance, the trigger time; we have to merge the information 
# (Year-Month-Day) from column 'GRB' with UT time from 'Time'.
#
import re
rec = re.compile('(\d+)?',re.IGNORECASE)
dt = pandas.DataFrame()
dt['day'] = swf.GRB.map(lambda s:'20{0}-{1}-{2}'.format(*re.findall('.{1,2}',rec.match(s).group(1))))

# We just saw that "Time" has null values. I understand we can
# fix this issue with a "01:01" time hour for what matters;
#
dt['time'] = swf['Time|[UT]'].fillna('01:01:01')
print dt.head(10)

          day         time
0  2017-03-25  07:56:57.95
1  2017-03-18     15:27:52
2  2017-03-18     12:11:56
3  2017-03-17     09:45:59
4  2017-03-11     08:08:42
5  2017-03-07     20:24:21
6  2017-03-06     07:06:12
7  2017-02-08     22:33:38
8  2017-02-08     18:11:16
9  2017-02-06     21:57:25


In [11]:
dt['trigger time(UT)'] = pandas.to_datetime( dt.apply(lambda x:' '.join([x.day,x.time]), axis=1), yearfirst=1, errors='raise')
# option 'coerce' enables the processing to complete even if some value was not able to be converted, in such cases
# 'NaT' ("null") is placed whenever convertion fails.
#
# That said, let's check whether the was any problem during conversion by verifying for Nulls
any(dt['trigger time(UT)'].isnull())

False

In [12]:
# As you may notice, dataframe 'dt' was created to ease the handling of datetime conversion.
# Column 'tt' has the timestamp right.
print dt.head()

          day         time        trigger time(UT)
0  2017-03-25  07:56:57.95 2017-03-25 07:56:57.950
1  2017-03-18     15:27:52 2017-03-18 15:27:52.000
2  2017-03-18     12:11:56 2017-03-18 12:11:56.000
3  2017-03-17     09:45:59 2017-03-17 09:45:59.000
4  2017-03-11     08:08:42 2017-03-11 08:08:42.000


In [13]:
# Let us now convert the position columns to degree units,
# they are currently in sexagesimal units.
dp = swf.filter(regex='(RA|Dec|Radius)')
dp_bat = dp.filter(regex='BAT')
dp_xrt = dp.filter(regex='XRT')
dp_uvot = dp.filter(regex='UVOT')

In [14]:
cols = dp_bat.columns
dp_bat[cols] = dp_bat[cols].apply(pandas.to_numeric, errors='ignore')
print dp_bat.head(10)
print dp_bat.describe()

   BAT RA|(J2000)  BAT Dec|(J2000) BAT 90%|Error Radius|[arcmin]
0         127.483           20.526                           2.0
1         284.229            6.335                           3.7
2         305.662           28.408                           1.2
3          93.087           50.500                           1.0
4         280.539          -30.041                           2.0
5          13.510            9.538                           2.5
6         263.087          -44.765                           2.4
7         127.145           -9.027                           1.0
8         166.544          -46.786                           1.4
9         271.784           11.175                           2.2
       BAT RA|(J2000)  BAT Dec|(J2000)
count     1122.000000      1122.000000
mean       184.190464         2.152165
std        104.756917        41.788303
min          0.127000       -89.029000
25%         93.541750       -29.822000
50%        188.518500         1.820500
75%        2

In [15]:
cols = dp_xrt.columns
dp_xrt[cols] = dp_xrt[cols].apply(pandas.to_numeric, errors='ignore')
print dp_xrt.head(10)
print dp_xrt.describe()

  XRT RA|(J2000) XRT Dec|(J2000)  XRT 90%|Error Radius|[arcsec]
0           None            None                            NaN
1    18:57:13.46      06:17:57.0                            1.8
2    20:22:40.12      28:24:21.7                            1.7
3    06:12:14.87      50:29:34.9                            1.4
4    18:42:21.46     -30:02:45.5                            1.5
5           None            None                            NaN
6    17:32:16.56     -44:44:53.5                            3.6
7    08:28:34.55     -09:01:47.7                            1.6
8    11:06:15.52     -46:46:05.6                            1.4
9    18:07:04.95      11:11:37.8                            1.6
       XRT 90%|Error Radius|[arcsec]
count                     897.000000
mean                        1.777077
std                         0.912496
min                         1.400000
25%                         1.400000
50%                         1.500000
75%                         1.700000


In [16]:
cols = dp_uvot.columns
dp_uvot[cols] = dp_uvot[cols].apply(pandas.to_numeric, errors='ignore')
print dp_uvot.head(10)
print dp_uvot.describe()

  UVOT RA|(J2000) UVOT Dec|(J2000) UVOT 90%|Error Radius|[arcsec]
0            None             None                           None
1            None             None                           None
2            None             None                           None
3     06:12:14.82       50:29:35.2                           0.43
4            None             None                           None
5            None             None                           None
6            None             None                           None
7            None             None                           None
8            None             None                           None
9            None             None                           None
       UVOT RA|(J2000) UVOT Dec|(J2000) UVOT 90%|Error Radius|[arcsec]
count              293              293                            272
unique             293              293                             47
top        20:37:01.80       36:18:08.8                      

In [17]:
# To transform those positions from sexagesimal to degrees we'll
# Astropy.
# We will try to fill most of the events with some (BAT,XRT,UVOT)
# positional information; we should try first the UVOT (better precision),
# if not available, XRT and finally we take (if available) the BAT position.
# If there is no positional information for a given GRB, "None" will
# be at the output.
# Also, to homogenize the final list of positions/error, Error Radius
# should be transformed to 'arcsec'.
#
from astropy.coordinates import SkyCoord
from astropy.units import Unit
ra,dec,rad = [],[],[]
inst = []
dd = pandas.DataFrame()
for i in swf.index:
    posinst = None
    if dp_uvot.iloc[i].values.all():
        posinst = 'UVOT'
        _ra,_dec,_rad = dp_uvot.iloc[i]
        _sc = SkyCoord(_ra,_dec,unit=('hourangle','deg'))
        _ra = _sc.ra.deg
        _dec = _sc.dec.deg
        try:
            _rad = _rad * Unit('arcsec')
            _rad = _rad.value
        except:
            _rad = None
    elif dp_xrt.iloc[i].values.all():
        posinst = 'XRT'
        _ra,_dec,_rad = dp_xrt.iloc[i]
        _sc = SkyCoord(_ra,_dec,unit=('hourangle','deg'))
        _ra = _sc.ra.deg
        _dec = _sc.dec.deg
        try:
            _rad = _rad * Unit('arcsec')
            _rad = _rad.value
        except:
            _rad = None
    elif dp_bat.iloc[i].values.all():
        posinst = 'BAT'
        _ra,_dec,_rad = dp_bat.iloc[i]
        _sc = SkyCoord(_ra,_dec,unit=('deg','deg'))
        _ra = _sc.ra.deg
        _dec = _sc.dec.deg
        try:
            _rad = _rad * Unit('arcmin')
            _rad = _rad.to('arcsec')
        except:
            print 'Error on radius: {}'.format(_rad)
            _rad = None
    else:
        _ra,_dec,_rad = None,None,None
    ra.append(_ra)
    dec.append(_dec)
    rad.append(_rad)
    inst.append(posinst)
dd['ra(deg)'] = ra
dd['dec(deg)'] = dec
dd['error radius(arcsec)'] = rad

Error on radius: TBD


In [18]:
# By now we should check whether tables have the same size to be clear that nothing was missing in the midway
assert len(dd) == len(swf)
assert len(dd) == len(dt)
print len(dd)

1123


In [19]:
dd = dd.join(dt['trigger time(UT)'])
print dd.describe()
print len(dd)

           ra(deg)     dec(deg)  error radius(arcsec)
count  1120.000000  1120.000000            846.000000
mean    184.454896     2.213356             33.223331
std     104.652577    41.757044             59.433465
min       0.127000   -89.008611              1.400000
25%      93.618531   -29.731347              1.500000
50%     188.798062     1.823028              1.700000
75%     275.911313    35.527569             60.000000
max     358.756750    88.610000            420.000000
1123


In [20]:
# Before joining the tables, we must to add our 'primary' column to this positional table -- the column we 
# will use to join the tables.
# The primary column is the 'GRB' name.
#
dd['GRB'] = swf['GRB']
cols = list(dd.columns)
cols = cols[-1:] + cols[:-1]
dd = dd[cols]
print dd

          GRB     ra(deg)   dec(deg)  error radius(arcsec)  \
0     170325A  127.483000  20.526000                 120.0   
1     170318B  284.306083   6.299167                   1.8   
2     170318A  305.667167  28.406028                   1.7   
3     170317A   93.061750  50.493111                   NaN   
4     170311A  280.589417 -30.045972                   1.5   
5     170307A   13.510000   9.538000                 150.0   
6     170306A  263.069000 -44.748194                   3.6   
7     170208B  127.143958  -9.029917                   1.6   
8     170208A  166.564667 -46.768222                   1.4   
9     170206B  271.770625  11.193833                   1.6   
10    170205A  262.169542  -0.063139                   NaN   
11    170203A  332.861000  25.186000                 132.0   
12    170202A  152.514542   5.011611                   NaN   
13    170131A  341.447000  64.006000                 180.0   
14    170127B   19.977083 -30.358083                   2.0   
15    17

In [21]:
# We may now try the JOIN.
# We want to join only the matching keys, the goal is to have `ibdhne` table upgraded with new (position,time)
# properties, but not to have more objects in it.
# To do this JOIN we define the 'primary' keys to be the indexes; pandas use it (the index) as the joining column.
#
ibd.set_index('GRB', inplace=True)
dd.set_index('GRB', inplace=True)

In [22]:
tab = ibd.join(dd, how='inner')
print tab.describe()

               z       E_iso           GCN     ra(deg)   dec(deg)  \
count  98.000000   98.000000     98.000000   98.000000  98.000000   
mean    2.277969   35.830306  13689.938776  182.851527   5.936088   
std     1.255099   58.487612   4135.107727  103.851478  44.172265   
min     0.338000    1.020000   3474.000000    2.053667 -79.128778   
25%     1.309750    5.792500  11343.000000  106.138052 -28.010576   
50%     2.087500   14.150000  14449.000000  187.342083   9.102153   
75%     3.030000   36.350000  16507.750000  273.550250  44.410854   
max     5.910000  347.000000  20192.000000  358.264375  87.301167   

       error radius(arcsec)  
count             38.000000  
mean               1.523684  
std                0.344403  
min                1.400000  
25%                1.400000  
50%                1.450000  
75%                1.500000  
max                3.500000  


We see that we lost almost half of our objects (98 out of 173). After some inspection on the side we easily get to the answer: the 'GRB' names are slightly different. The reason is due to the name convention of those events: because the GRB name is built from the date it was observed if more then one event is detect at the same day, alpha character (A,B,C,...) are appended to the end of the name; Some tables will append an '**A**' to the name event if only one event happens to be detected on a given day (to be consistent). And *that* is the reason for unmatching keys in our JOIN.

In [23]:
# to solve the unmatching keys we'll use Python's (stdlib) 'difflib'
from difflib import get_close_matches
diff = ibd.reset_index().GRB.to_frame()
diff['match'] = diff.GRB.map(lambda x:get_close_matches(x, dd.index, 1)[0])
# to double-check if we are right, the following column 'equal' must sum '98'
diff['equal'] = diff['GRB'] == diff['match']
print sum(diff.equal)

98


In [24]:
diff

,GRB,match,equal
0,050315A,050315,False
1,050318A,050318,False
2,050319A,050319,False
3,050401A,050401,False
4,050408A,051008,False
5,050505A,050505,False
6,050525A,050525A,True
7,050730A,050730,False
8,050802A,050802,False
9,050814A,050814,False


This defines our new index/primary key -- the so called 'match' column.
We now redefine it to 'ibd' and go back to the business...

In [25]:
ibd.reset_index(inplace=True)
ibd['primary'] = ibd.GRB.map(lambda x:get_close_matches(x,dd.index,1)[0])
ibd.set_index('primary', inplace=True)

In [26]:
tab = ibd.join(dd, how='inner')
print tab.describe()

                z       E_iso           GCN     ra(deg)    dec(deg)  \
count  173.000000  173.000000    173.000000  173.000000  173.000000   
mean     2.244382   40.616358  11356.485549  178.858973    2.553452   
std      1.264709   69.514891   4999.373872  102.756476   42.041703   
min      0.338000    1.020000   3099.000000    0.913333  -79.128778   
25%      1.314000    6.150000   7289.000000   89.275625  -29.613639   
50%      2.060000   15.000000  11169.000000  183.501167    2.185000   
75%      2.993000   36.500000  15796.000000  258.981208   35.515944   
max      8.200000  443.000000  20192.000000  358.264375   87.301167   

       error radius(arcsec)  
count             78.000000  
mean               5.615385  
std               19.496054  
min                1.400000  
25%                1.400000  
50%                1.400000  
75%                1.500000  
max              144.000000  


Now we have it! ;)

That's it! Well...amost... Let's now clean/homogenize column names and wirte it down to CSV file.

In [27]:
print tab.columns.values

['GRB' 'designation' 'z' 'E_iso' 'instrument' 'GCN' 'ra(deg)' 'dec(deg)'
 'error radius(arcsec)' 'trigger time(UT)']


In [28]:
tab.rename(columns={'E_iso':'E_iso(1e52 erg)'}, inplace=True)

In [29]:
tab['ra(deg)'] = tab['ra(deg)'].round(3)
tab['dec(deg)'] = tab['dec(deg)'].round(3)
tab['error radius(arcsec)'] = tab['error radius(arcsec)'].round(3)

In [30]:
print tab.describe(include='all')

            GRB     designation           z  E_iso(1e52 erg) instrument  \
count       173             173  173.000000       173.000000        173   
unique      173             173         NaN              NaN          5   
top     081109A  IBdHNe 120327A         NaN              NaN      Fermi   
freq          1               1         NaN              NaN         62   
first       NaN             NaN         NaN              NaN        NaN   
last        NaN             NaN         NaN              NaN        NaN   
mean        NaN             NaN    2.244382        40.616358        NaN   
std         NaN             NaN    1.264709        69.514891        NaN   
min         NaN             NaN    0.338000         1.020000        NaN   
25%         NaN             NaN    1.314000         6.150000        NaN   
50%         NaN             NaN    2.060000        15.000000        NaN   
75%         NaN             NaN    2.993000        36.500000        NaN   
max         NaN          

In [31]:
# Last step to be taken here is to fill the missing values for 'error radius'.
# The safest way to do it is to fill the Nulls with the maximum possible value,
# doing so we will not give absurd, or erronous information to the future users
# of such data since we are filling those gaps with the worst case available.
mval = tab['error radius(arcsec)'].max()
midx = tab['error radius(arcsec)'].isnull()
tab.loc[midx,'error radius(arcsec)'] = mval
print mval
tab.describe()

144.0


,z,E_iso(1e52 erg),GCN,ra(deg),dec(deg),error radius(arcsec)
count,173.000000,173.000000,173.000000,173.000000,173.000000,173.000000
mean,2.244382,40.616358,11356.485549,178.858965,2.553451,81.606936
std,1.264709,69.514891,4999.373872,102.756458,42.041692,70.278524
min,0.338000,1.020000,3099.000000,0.913000,-79.129000,1.400000
25%,1.314000,6.150000,7289.000000,89.276000,-29.614000,1.500000
50%,2.060000,15.000000,11169.000000,183.501000,2.185000,144.000000
75%,2.993000,36.500000,15796.000000,258.981000,35.516000,144.000000
max,8.200000,443.000000,20192.000000,358.264000,87.301000,144.000000


In [32]:
tab.to_csv('data_wposition.csv',sep=';', index=False)